## Submission Extraction ZST File 

- This code was ran on Google Colab, as explained in ReadMe. The main part of the code which extracts and decodes the information is from the creator of the files and available here: https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/single_file.py. It is also cited in my ReadMe.

In [ ]:
import zstandard
import json
import csv
from datetime import datetime

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Conservative_submissions.zst to Conservative_submissions (1).zst


In [ ]:
# code from watchful1 github
# Define function to read and decode chunks
def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
    chunk = reader.read(chunk_size)
    bytes_read += len(chunk)
    if previous_chunk is not None:
        chunk = previous_chunk + chunk
    try:
        return chunk.decode()
    except UnicodeDecodeError:
        if bytes_read > max_window_size:
            raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
        log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
        return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)

# Define function to read lines from Zstandard compressed file
def read_lines_zst(file_name):
    with open(file_name, 'rb') as file_handle:
        buffer = ''
        reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
        while True:
            chunk = read_and_decode(reader, 2**27, (2**29) * 2)

            if not chunk:
                break
            lines = (buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            buffer = lines[-1]

        reader.close()

# Example usage:
file_name = 'Conservative_submissions.zst'  # Replace with your file path
num_lines_to_print = 5  # Number of lines to print

# Iterate over the first few lines and print them
for i, line in enumerate(read_lines_zst(file_name)):
    print(line)
    if i == num_lines_to_print - 1:
        break

{"archived":true,"author":"[deleted]","author_flair_background_color":"","author_flair_css_class":null,"author_flair_text":null,"author_flair_text_color":"dark","brand_safe":true,"can_gild":false,"contest_mode":false,"created_utc":1202154642,"distinguished":null,"domain":"amconmag.com","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"67n0c","is_crosspostable":false,"is_reddit_media_domain":false,"is_self":false,"is_video":false,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"no_follow":true,"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"\/r\/Conservative\/comments\/67n0c\/the_great_betrayal_pat_buchanan_on_john_mccain\/","retrieved_on":1522683460,"rte_mode":"markdown","score":1,"secure_media":null,"secure_media_embed":{},"selftext":"[deleted]","send_replies":true,"spoiler":false,"stickied

In [ ]:
file_name = 'Conservative_submissions.zst'

In [ ]:
# FUNC FOR GETTING INFO
def extract_submission_info(submission):
  readable_dt = datetime.utcfromtimestamp(int(submission['created_utc'])).strftime('%Y-%m-%d %H:%M:%S')
  return {
        'title': submission['title'],
        'date_posted': readable_dt,
        'author': submission['author'],
        'id': submission['id'],
        'flare': submission['link_flair_text'],
        'num_comments': submission['num_comments'],
        'url': submission['permalink']
    }


In [ ]:
# Output CSV file for all submissions
output_file_all = 'cons_submissions_all.csv'

# Output CSV file for abortion posts
output_file_abortion = 'cons_submissions_abortion_retry_dn.csv'


In [ ]:
# Actual code to run
for i, line in enumerate(read_lines_zst(file_name)):
    submission = json.loads(line)
    with open(output_file_all, 'a') as f_all, open(output_file_abortion, 'a') as f_abortion:
        fieldnames = ['title', 'date_posted', 'author', 'id', 'flare', 'num_comments', 'url']
        writer_all = csv.DictWriter(f_all, fieldnames=fieldnames)
        writer_abortion = csv.DictWriter(f_abortion, fieldnames=fieldnames)

        if i == 0:
            writer_all.writeheader()
            writer_abortion.writeheader()

        submission_info = extract_submission_info(submission)

        writer_all.writerow(submission_info)

        # part where abortion is written to new one
        if 'abortion' in submission['title'].lower():
            writer_abortion.writerow(submission_info)

    # if i == num_lines_to_print - 1:
    #     break

In [ ]:
import pandas as pd

cons_all = pd.read_csv('cons_submissions_all.csv')

cons_all.head(20)

,title,date_posted,author,id,flare,num_comments,url
0,The Great Betrayal: Pat Buchanan on John McCain,2008-02-04 19:50:42,[deleted],67n0c,NaN,0,/r/Conservative/comments/67n0c/the_great_betra...
1,Gary Becker: Selling illegal immigrants the ri...,2008-02-04 19:51:25,[deleted],67n0j,NaN,0,/r/Conservative/comments/67n0j/gary_becker_sel...
2,Searing Vision Naivete: Analyzing Toni Morriso...,2008-02-04 20:00:11,[deleted],67n1u,NaN,0,/r/Conservative/comments/67n1u/searing_vision_...
3,"Thomas Sowell - ""The Education of Minority Chi...",2008-02-04 20:48:55,[deleted],67n8x,NaN,0,/r/Conservative/comments/67n8x/thomas_sowell_t...
4,"Hillary Clinton went to all-female Wellesley, ...",2008-02-04 23:04:29,[deleted],67npy,NaN,0,/r/Conservative/comments/67npy/hillary_clinton...
5,Political Cartoons 2-12-08,2008-02-13 11:20:23,unkorrupted,68lfw,NaN,0,/r/Conservative/comments/68lfw/political_carto...
6,Ron Paul and Raw Milk,2008-03-06 02:38:58,temjrpgh,6b1e5,NaN,1,/r/Conservative/comments/6b1e5/ron_paul_and_ra...
7,The Vietnam Veterans' Memorial and Our Politic...,2008-06-14 17:57:44,[deleted],6ndpk,NaN,0,/r/Conservative/comments/6ndpk/the_vietnam_vet...
8,Is Democracy Feasible? Reflections on the Gett...,2008-06-25 05:34:14,[deleted],6ouxd,NaN,0,/r/Conservative/comments/6ouxd/is_democracy_fe...
9,Obama vs. Europe,2008-06-30 20:31:34,michael123,6pnvt,NaN,0,/r/Conservative/comments/6pnvt/obama_vs_europe/
